In [2]:
import os

from mlxtend.classifier import LogisticRegression

from data.teamExperiments.trainClassifiers import trainStackingClassifier

os.chdir('/Users/thom/Personnel/Scolaire/MIA/snake2-beta-insa-main/data/teamExperiments')

from trainClassifiers import *
from prepareData import *

In [13]:
print("\n--------------------------------\nCreation of classifiers training dataset\n--------------------------------\n")
%run classification/makeClassifierTrainingDataset2.ipynb
print("\n--------------------------------\nTraining of classifiers\n--------------------------------\n")


----------------
Creation of classifiers training dataset
----------------


----------------
Training of classifiers
----------------



In [14]:
X_train, X_test, y_train, y_test = get_train_test_X_y(trainingSet, 'isMember', size=0.3)
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [4]:
modelsToUse = [
    LogisticRegression,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier,
    xgboost.XGBClassifier,
]

LogisticRegression


In [16]:
classifiersCollection = trainModels(modelsToUse, X_train, y_train)

Training model : <class 'sklearn.linear_model._logistic.LogisticRegression'>
Training model : <class 'sklearn.naive_bayes.GaussianNB'>


In [17]:
predictionsCollection = createCollection_Predictions(classifiersCollection, X_test)
precisionsCollection = createCollection_PrecisionScore(y_test, predictionsCollection)
ROCMCollection = createCollection_ROC(classifiersCollection, X, y, X_train, y_train, value = "mean")
ROCSCollection = createCollection_ROC(classifiersCollection, X, y, X_train, y_train, value = "std")
confusionMatricesCollection = createCollection_ConfusionMatrices(y_test, predictionsCollection)

Computing ROC for : LogisticRegression()
Computing ROC for : GaussianNB()
Computing ROC for : LogisticRegression()
Computing ROC for : GaussianNB()


In [19]:
classifiers_MetricsDatabase = createClassifiersMetricsDB(classifiersCollection, X, y, precisionsCollection, confusionMatricesCollection, ROCMCollection, ROCSCollection)

Adding model to DB : LogisticRegression()
Adding model to DB : GaussianNB()


### Adding Stacking Classifier


In [20]:
StackClass = trainStackingClassifier(classifiersCollection, X_train, y_train)
StackArray = [StackClass]
predictionsCollection = createCollection_Predictions(StackArray, X_test)
precisionsCollection = createCollection_PrecisionScore(y_test, predictionsCollection)
ROCMCollection = createCollection_ROC(StackArray, X, y, X_train, y_train, value = "mean")
ROCSCollection = createCollection_ROC(StackArray, X, y, X_train, y_train, value = "std")
confusionMatricesCollection = createCollection_ConfusionMatrices(y_test, predictionsCollection)

classifiers_MetricsDatabase.loc[len(classifiers_MetricsDatabase)] = (StackClass , StackClass.__class__.__name__, StackClass.score(X,y), precisionsCollection[0], str((confusionMatricesCollection[0]).tolist()), ROCMCollection[0], ROCSCollection[0])

classifiersCollection.append(StackClass)

Computing ROC for : StackingClassifier(classifiers=[LogisticRegression(), GaussianNB()],
                   meta_classifier=LogisticRegression())
Computing ROC for : StackingClassifier(classifiers=[LogisticRegression(), GaussianNB()],
                   meta_classifier=LogisticRegression())
[array([[0.27, 0.24],
       [0.27, 0.22]])]


In [22]:
directory_path = f"/Users/thom/Personnel/Scolaire/MIA/snake2-beta-insa-main/data/teamExperiments/results/classifiersPerfs/"
file_name = "newClassifiers_"
saveClassifiersMetricsDB(classifiers_MetricsDatabase, directory_path, file_name)